### Install dependencies

In [20]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [21]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### Instantiate Model

In [23]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Encode and Calculate Sentiment

In [24]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [25]:
result = model(tokens)

In [26]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [27]:
int(torch.argmax(result.logits))+1

4

### Collect Reviews from a website

In [28]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [29]:
reviews

['Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.",
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.',
 "It was ok. The coffee wasn't the best but it was fine. The relish on the brea

### Load Reviews into DataFrame and Score

In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [32]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [33]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [34]:
sentiment_score(df['review'].iloc[1])

3

In [35]:
#df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [36]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [37]:
df

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5
5,It was ok. The coffee wasn't the best but it w...,3
6,We came for brunch twice in our week-long visi...,4
7,I went here a little while ago- a beautiful mo...,2
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4


#### Note:  This is referred partly from Nick youtube video